In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
from astropy.table import Table,vstack
from desiutil.log import get_logger, DEBUG
from desidiff.src.group_tiles import *
from desidiff.src.dates_to_process import *
from desidiff.src.coadd import *
from desidiff.src.scores import *

from timedomain.sp_utils import SkyPortal as sp
import requests

In [ ]:
# Loop over unprocessed night, doesn't need to be run as we debug
night_arr = getUnprocessedDates()
for a in night_arr:   
    # read in and store in one place all the fibermap information in the spectra files
    date = a

In [ ]:
date=20210606
tile_petal,group_tid,group_tp, group_night =getMatchedTileid(date)

In [ ]:
#Set non-default plot size 
plt.rcParams["figure.figsize"] = (20,6)

In [ ]:
# tile_petal contain subsets of tile/petals whose RA/DEC targets are not contained in other tile/petals from that night
for tps in tile_petal:
    # Check to see if this tile/petal has nothing to subtract.  Skip's IO if this is the case
    if hasNothingToProcess(tps,group_tid,group_tp,group_night):
        continue
    
    # cache spectra to minimize IO
    spectra_list=dict()
    zbest_list = []
    target_list=[]
    for tp in tps:
        filename =  f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{tp[0]}/{date}/spectra-{tp[1]}-{tp[0]}-thru{date}.fits"
        spectra_list[(tp[0],tp[1])]=read_spectra(filename)
        zbest = filename.replace('spectra','zbest')
        z = Table.read(zbest, format='fits',hdu=1, memmap=True)['TARGETID','Z','ZERR','ZWARN','SPECTYPE']
        zbest_list.append(z)
    z = vstack(zbest_list)
    

#     z_data = []
    count=0
    # loop over all unique RA/DEC pairs from that night
    for tid, tp, night in zip(group_tid,group_tp,group_night):
        # if this RA/DEC is not in thie tile_petal combination than skip
        if tp[0] not in tps:
            continue
            
        if len(night) == 1:
            continue
            
#         print(tid)

        # Eliminate those with no reference night here
        # Proceed with a subtraction for this object
        
        # The coadds of the new and reference are constructed from all spectra with all targetid's in tid and all
        # tile/petal combinations in tp, which are cached above
        
        # Finds the object in the zbest by matching the targetID.
        #Stores a tuple = (TARGETID, Z, ZERR,ZWARN, SPECTYPE)
        z_data = []
        for t in tid:
            tid_ind = list(z['TARGETID']).index(t)
            z_data.append((tid, z[tid_ind]['Z'], z[tid_ind]['ZERR'], z[tid_ind]['ZWARN'], z[tid_ind]['SPECTYPE']))
        
        newSpectra=[]
        refSpectra=[]
        for tile,plate in tp:
            spec = spectra_list[(tile,plate)]

            idx = numpy.in1d(night, date)
            ref_night = night[~idx]
            
            newSpectra.append(spec.select(nights=date, targets = tid))
            
            """
            There is a variable night that has all the nights associated with this RA/DEC
            derive the ref_night from that
            """
            
            refSpectra.append(spec.select(nights=ref_night, targets = tid))       
         
        
        newflux, newivar, newwave, newmask = coadd(newSpectra)
        refflux, refivar, refwave, refmask = coadd(refSpectra)
        
        difflux, difivar, difmask, difwave = dict.fromkeys(["b", "r", "z"]), dict.fromkeys(["b", "r", "z"]), dict.fromkeys(["b", "r", "z"]), dict.fromkeys(["b", "r", "z"])
        
        difflux = {key: newflux[key] - refflux[key]
                       for key in newflux.keys()}
        difivar = {key: newivar[key] + refivar[key]
                       for key in newivar.keys()}
        difmask = {key: newmask[key] + refmask[key]
                       for key in newmask.keys()}
        difwave = dict(newwave)

        ## filters         
        # trim red edge 
        if 'z' in difflux.keys():
            w=numpy.where(difwave['z'] < 9100)[0]
            difflux['z']=difflux['z'][w]
            difivar['z']=difivar['z'][w]
            difwave['z']=difwave['z'][w]
            difmask['z']=difmask['z'][w]
            
        # mean-subtracted difference
        difflux_clipmean = clipmean(difflux,difivar,difmask)
        
        # Difference spectrum may have broadband signal
        perband_filter = perband_SN(difflux_clipmean,difivar,difmask)
        
        # fractional increase
        perband_inc = perband_increase(difflux_clipmean,difivar,difmask, refflux,refivar,refmask)
        
        # Difference spectrum may have high-frequency signal
        perres_filter = perconv_SN(difflux_clipmean,difivar,difmask)
        
#         print(perband_filter, perband_inc, perres_filter)
#         print(any(list(perband_filter.values())), numpy.sum(list(perres_filter.values())) >1)

        #broadband
        bblogic = any(numpy.logical_and(numpy.array(list(perband_filter.values()))>10, numpy.array(list(perband_inc.values()))>0.25))
        linelogic = numpy.sum(list(perres_filter.values()))>4
        logic = bblogic or linelogic
        plt.clf()
        if logic:
            #Uncomment next line if you want to print only those TargetIds that get plotted
            print(tid)
            for b in difflux.keys():
                w=numpy.where(difmask[b] ==0)[0]
                if b == list(difflux.keys())[-1]:
                    plt.plot(difwave[b][w],difflux[b][w],color='cyan', label = 'Difference')
                    plt.plot(newwave[b][w],newflux[b][w],color='blue',alpha=0.5, label = 'New Spectrum')
                    plt.plot(refwave[b][w],refflux[b][w],color='green',alpha=0.5, label = 'Reference Spectrum')
                    plt.legend()
                else:
                    plt.plot(difwave[b][w],difflux[b][w],color='cyan')
                    plt.plot(newwave[b][w],newflux[b][w],color='blue',alpha=0.5)
                    plt.plot(refwave[b][w],refflux[b][w],color='green',alpha=0.5)

            plt.xlabel('Wavelength (A)')
            plt.ylabel('Flux') 
            plt.show()
            count=count+1
            
        if count==10:
            wefwe

In [38]:
token = 'b8755e6b-e63c-410e-8bcf-45debfbf363a'

In [39]:
def api(method, endpoint, data=None):
    headers = {'Authorization': f'token {token}'}
    response = requests.request(method, endpoint, json=data, headers=headers)
    return response

In [40]:
wavelengths = (numpy.concatenate((difwave['b'],difwave['r'],difwave['z']), axis=None))
wavelengths = wavelengths.tolist()
# print(wavelengths)

In [41]:
flux = (numpy.concatenate((difflux['b'],difflux['r'],difflux['z']), axis=None))
flux = flux.tolist()
# print(flux)

In [ ]:
difspectrum = Spectra(list(difflux.keys()), difwave, difflux, difivar, difmask)

In [67]:
print(sp.instrument_id())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [42]:
data = {
"obj_id": str(tid[0]),
"wavelengths": wavelengths,
"fluxes": flux,
"observed_at": str(date),
"instrument_id": 7
}


In [ ]:
print(data)

In [43]:
response = api('POST', '{}/api/spectrum'.format(sp.url), data=data)

In [44]:
print(response.content)

b'{"status": "error", "message": "HTTP 405: Method Not Allowed", "data": {}}'


In [53]:
import datetime
from astropy.time import Time
d = datetime.datetime(2014, 12, 10, 12, 0, 0)
str(d).replace('+00:00', 'Z')

'2014-12-10 12:00:00'

In [65]:
token = "85c366c9-88aa-47bc-81c8-3a19366d9789"
response = requests.post(
    "http://desi2.lbl.gov:5000/api/spectrum",
    json={
        "obj_id": str(tid[0]),
"wavelengths": wavelengths,
"fluxes": flux,
"observed_at": str(d),
"instrument_id": 7
    },
    headers={"Authorization": f"token {token}"}
)

In [66]:
print(response.content)

b'{"status": "error", "message": "(psycopg2.errors.ForeignKeyViolation) insert or update on table \\"spectra\\" violates foreign key constraint \\"spectra_obj_id_fkey\\"\\nDETAIL:  Key (obj_id)=(39632967152177739) is not present in table \\"objs\\".\\n\\n[SQL: INSERT INTO spectra (created_at, modified, wavelengths, fluxes, errors, obj_id, observed_at, origin, instrument_id, followup_request_id, assignment_id, original_file_string, original_file_filename, owner_id) VALUES (timezone(%(timezone_1)s, CURRENT_TIMESTAMP), timezone(%(timezone_1)s, CURRENT_TIMESTAMP), %(wavelengths)s, %(fluxes)s, %(errors)s, %(obj_id)s, %(observed_at)s, %(origin)s, %(instrument_id)s, %(followup_request_id)s, %(assignment_id)s, %(original_file_string)s, %(original_file_filename)s, %(owner_id)s) RETURNING spectra.id]\\n[parameters: {\'timezone_1\': \'UTC\', \'wavelengths\': [3600.0, 3600.8, 3601.6000000000004, 3602.4000000000005, 3603.2000000000007, 3604.000000000001, 3604.800000000001, 3605.6000000000013, 3606.